<div>

> **Note**
>
> Code chunks run Python commands unless it starts with `%%bash`, in
> which case, those chunks run shell commands.

</div>

Celltype prediction can either be performed on indiviudal cells where
each cell gets a predicted celltype label, or on the level of clusters.
All methods are based on similarity to other datasets, single cell or
sorted bulk RNAseq, or uses known marker genes for each cell type.\
Ideally celltype predictions should be run on each sample separately and
not using the integrated data. In this case we will select one sample
from the Covid data, `ctrl_13` and predict celltype by cell on that
sample.\
Some methods will predict a celltype to each cell based on what it is
most similar to, even if that celltype is not included in the reference.
Other methods include an uncertainty so that cells with low similarity
scores will be unclassified.\
There are multiple different methods to predict celltypes, here we will
just cover a few of those.

Here we will use a reference PBMC dataset that we get from scanpy
datasets and classify celltypes based on two methods:

-   Using scanorama for integration just as in the integration lab, and
    then do label transfer based on closest neighbors.
-   Using ingest to project the data onto the reference data and
    transfer labels.
-   Using Celltypist to predicted with a pretrained pbmc model or with
    an own model based on the same reference data as the other methods.

First, lets load required libraries

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import warnings
import os
import urllib.request

warnings.simplefilter(action="ignore", category=Warning)

# verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.verbosity = 2
sc.settings.set_figure_params(dpi=80)

Let's read in the saved Covid-19 data object from the clustering step.

In [ ]:
# download pre-computed data if missing or long compute
fetch_data = True

# url for source and intermediate data
path_data = "https://export.uppmax.uu.se/naiss2023-23-3/workshops/workshop-scrnaseq"

path_results = "data/covid/results"
if not os.path.exists(path_results):
    os.makedirs(path_results, exist_ok=True)

# path_file = "data/covid/results/scanpy_covid_qc_dr_scanorama_cl.h5ad"
path_file = "data/covid/results/scanpy_covid_qc_dr_scanorama_cl.h5ad"
if fetch_data and not os.path.exists(path_file):
    urllib.request.urlretrieve(os.path.join(
        path_data, 'covid/results/scanpy_covid_qc_dr_scanorama_cl.h5ad'), path_file)

adata = sc.read_h5ad(path_file)
adata

In [ ]:
adata.uns['log1p']['base']=None
print(adata.shape)

Subset one patient.

In [ ]:
adata = adata[adata.obs["sample"] == "ctrl_13",:]
print(adata.shape)

In [ ]:
adata.obs["leiden_0.6"].value_counts()

Some clusters have very few cells from this individual, so any cluster
comparisons may be biased by this.

In [ ]:
sc.pl.umap(
    adata, color=["leiden_0.6"], palette=sc.pl.palettes.default_20
)

## Reference data

Load the reference data from `scanpy.datasets`. It is the annotated and
processed pbmc3k dataset from 10x.

In [ ]:
adata_ref = sc.datasets.pbmc3k_processed() 

adata_ref.obs['sample']='pbmc3k'

print(adata_ref.shape)
adata_ref.obs

As you can see, the celltype annotation is in the metadata column
`louvain`, so that is the column we will have to use for classification.

In [ ]:
sc.pl.umap(adata_ref, color='louvain')

Make sure we have the same genes in both datset by taking the
intersection

In [ ]:
# before filtering genes, store the full matrix in raw.
adata.raw = adata
# also store the umap in a new slot as it will get overwritten
adata.obsm["X_umap_uncorr"] = adata.obsm["X_umap"]

print(adata_ref.shape[1])
print(adata.shape[1])
var_names = adata_ref.var_names.intersection(adata.var_names)
print(len(var_names))

adata_ref = adata_ref[:, var_names]
adata = adata[:, var_names]

First we need to rerun pca and umap with the same gene set for both
datasets.

In [ ]:
sc.pp.pca(adata_ref)
sc.pp.neighbors(adata_ref)
sc.tl.umap(adata_ref)
sc.pl.umap(adata_ref, color='louvain')

In [ ]:
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.pl.umap(adata, color='leiden_0.6')

## Integrate with scanorama

In [ ]:
import scanorama

#subset the individual dataset to the same variable genes as in MNN-correct.
alldata = dict()
alldata['ctrl']=adata
alldata['ref']=adata_ref

#convert to list of AnnData objects
adatas = list(alldata.values())

# run scanorama.integrate
scanorama.integrate_scanpy(adatas, dimred = 50)

In [ ]:
# add in sample info
adata_ref.obs['sample']='pbmc3k'

# create a merged scanpy object and add in the scanorama 
adata_merged = alldata['ctrl'].concatenate(alldata['ref'], batch_key='sample', batch_categories=['ctrl','pbmc3k'])

embedding = np.concatenate([ad.obsm['X_scanorama'] for ad in adatas], axis=0)
adata_merged.obsm['Scanorama'] = embedding

In [ ]:
#run  umap.
sc.pp.neighbors(adata_merged, n_pcs =50, use_rep = "Scanorama")
sc.tl.umap(adata_merged)

In [ ]:
sc.pl.umap(adata_merged, color=["sample","louvain"])

### Label transfer

Using the functions from the [Spatial
tutorial](https://scanpy.readthedocs.io/en/stable/tutorials/spatial/integration-scanorama.html)
from Scanpy we will calculate normalized cosine distances between the
two datasets and tranfer labels to the celltype with the highest scores.

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

distances = 1 - cosine_distances(
    adata_merged[adata_merged.obs['sample'] == "pbmc3k"].obsm["Scanorama"],
    adata_merged[adata_merged.obs['sample'] == "ctrl"].obsm["Scanorama"],
)

def label_transfer(dist, labels, index):
    lab = pd.get_dummies(labels)
    class_prob = lab.to_numpy().T @ dist
    norm = np.linalg.norm(class_prob, 2, axis=0)
    class_prob = class_prob / norm
    class_prob = (class_prob.T - class_prob.min(1)) / class_prob.ptp(1)
    # convert to df
    cp_df = pd.DataFrame(
        class_prob, columns=lab.columns
    )
    cp_df.index = index
    # classify as max score
    m = cp_df.idxmax(axis=1)
    
    return m

class_def = label_transfer(distances, adata_ref.obs.louvain, adata.obs.index)

# add to obs section of the original object
adata.obs['label_trans'] = class_def

sc.pl.umap(adata, color="label_trans")

In [ ]:
# add to merged object.
adata_merged.obs["label_trans"] = pd.concat(
    [class_def, adata_ref.obs["louvain"]], axis=0
).tolist()

sc.pl.umap(adata_merged, color=["sample","louvain",'label_trans'])
#plot only ctrl cells.
sc.pl.umap(adata_merged[adata_merged.obs['sample']=='ctrl'], color='label_trans')

Now plot how many cells of each celltypes can be found in each cluster.

In [ ]:
tmp = pd.crosstab(adata.obs['leiden_0.6'],adata.obs['label_trans'], normalize='index')
tmp.plot.bar(stacked=True).legend(bbox_to_anchor=(1.8, 1),loc='upper right')

## Ingest

Another method for celltype prediction is Ingest, for more information,
please look at
https://scanpy-tutorials.readthedocs.io/en/latest/integrating-data-using-ingest.html

In [ ]:
sc.tl.ingest(adata, adata_ref, obs='louvain')
sc.pl.umap(adata, color=['louvain','leiden_0.6'], wspace=0.5)

As you can see, ingest has created a new umap for us, so to get
consistent plotting, lets revert back to the old one for further
plotting:

In [ ]:
adata.obsm["X_umap"] = adata.obsm["X_umap_uncorr"]

sc.pl.umap(adata, color=['louvain','leiden_0.6'], wspace=0.5)

Now plot how many cells of each celltypes can be found in each cluster.

In [ ]:
tmp = pd.crosstab(adata.obs['leiden_0.6'],adata.obs['louvain'], normalize='index')
tmp.plot.bar(stacked=True).legend(bbox_to_anchor=(1.8, 1),loc='upper right')

## Celltypist

[Celltypist](https://www.celltypist.org/) provides pretrained models for
classification for many different human tissues and celltypes. Here, we
are following the steps of this
[tutorial](https://colab.research.google.com/github/Teichlab/celltypist/blob/main/docs/notebook/celltypist_tutorial.ipynb),
with some adaptations for this dataset. So please check out the tutorial
for more detail.

In [ ]:
import celltypist
from celltypist import models

# there are many different models, we will only download 2 of them for now.
models.download_models(force_update = False, model = 'Immune_All_Low.pkl')
models.download_models(force_update = False, model = 'Immune_All_High.pkl')

Now select the model you want to use and show the info:

In [ ]:
model = models.Model.load(model = 'Immune_All_High.pkl')

model

To infer celltype labels to our cells, we first need to convert back to
the full matrix. OBS! For celltypist we want to have log1p normalised
expression to 10,000 counts per cell. Which we already have in
`adata.raw.X`, check by summing up the data, it should sum to 10K.

In [ ]:
adata = adata.raw.to_adata() 
adata.X.expm1().sum(axis = 1)[:10]

In [ ]:
predictions = celltypist.annotate(adata, model = 'Immune_All_High.pkl', majority_voting = True)

predictions.predicted_labels

The first column `predicted_labels` is the predictions made for each
individual cell, while `majority_voting` is done for local subclusters,
the clustering identities are in column `over_clustering`.

Now we convert the predictions to an anndata object.

In [ ]:
adata = predictions.to_adata()

sc.pl.umap(adata, color = ['leiden_0.6', 'predicted_labels', 'majority_voting'], legend_loc = 'on data')

<div>

> **Task**
>
> Rerun predictions with Celltypist, but use another model, for instance
> `Immune_All_High.pkl`, or any other model you find relevant, you can
> find a list of models [here](https://www.celltypist.org/models). How
> do the results differ for you?

</div>

### Celltypist custom model

We can also train our own model on any reference data that we want to
use. In this case we will use the pbmc data in `adata_ref` to train a
model.

Celltypist requires the data to be in the format of log1p normalised
expression to 10,000 counts per cell, we can check if that is the case
for the object we have:

In [ ]:
adata_ref.raw.X.expm1().sum(axis = 1)[:10]

These should all sum up to 10K, which is not the case, probably since
some genes were removed after normalizing. Wo we will have to start from
the raw counts of that dataset instead. Before we selected the data
`pbmc3k_processed`, but now we will instead use `pbmc3k`.

In [ ]:
adata_ref2 = sc.datasets.pbmc3k() 
adata_ref2

This data is not annotated, so we will have to match the indices from
the filtered and processed object. And add in the metadata with
annotations.

In [ ]:
adata_ref2 = adata_ref2[adata_ref.obs_names,:]
adata_ref2.obs = adata_ref.obs
adata_ref2

Now we can normalize the matrix:

In [ ]:
sc.pp.normalize_total(adata_ref2, target_sum = 1e4)
sc.pp.log1p(adata_ref2)

# check the sums again
adata_ref2.X.expm1().sum(axis = 1)[:10]

And finally train the model.

In [ ]:
new_model = celltypist.train(adata_ref2, labels = 'louvain', n_jobs = 10, feature_selection = True)

Now we can run predictions on our data

In [ ]:
predictions2 = celltypist.annotate(adata, model = new_model, majority_voting = True)

Instead of converting the predictions to anndata we will just add
another column in the `adata.obs` with these new predictions since the
column names from the previous celltypist runs with clash.

In [ ]:
adata.obs["predicted_labels_ref"] = predictions2.predicted_labels["predicted_labels"]
adata.obs["majority_voting_ref"] = predictions2.predicted_labels["majority_voting"]

In [ ]:
sc.pl.umap(adata, color = ['predicted_labels', 'majority_voting','predicted_labels_ref', 'majority_voting_ref'], legend_loc = 'on data', ncols=2)

## Compare results

The predictions from ingest is stored in the column 'louvain' while we
named the label transfer with scanorama as 'predicted'

In [ ]:
sc.pl.umap(adata, color=['louvain','label_trans','majority_voting', 'majority_voting_ref'], wspace=0.5, ncols=3)

As you can see, the main celltypes are generally the same, but there are
clearly differences, especially with regards to the cells predicted as
either ILC/NK/CD8 T-cells.

The only way to make sure which method you trust is to look at what
genes the different celltypes express and use your biological knowledge
to make decisions.

## Gene set analysis

Another way of predicting celltypes is to use the differentially
expressed genes per cluster and compare to lists of known cell marker
genes. This requires a list of genes that you trust and that is relevant
for the tissue you are working on.

You can either run it with a marker list from the ontology or a list of
your choice as in the example below.

In [ ]:
path_file = 'data/human_cell_markers.txt'
if not os.path.exists(path_file):
    urllib.request.urlretrieve(os.path.join(
        path_data, 'human_cell_markers.txt'), path_file)

In [ ]:
df = pd.read_table(path_file)
df

print(df.shape)

In [ ]:
# Filter for number of genes per celltype
df['nG'] = df.geneSymbol.str.split(",").str.len()

df = df[df['nG'] > 5]
df = df[df['nG'] < 100]
d = df[df['cancerType'] == "Normal"]
print(df.shape)

In [ ]:
df.index = df.cellName
gene_dict = df.geneSymbol.str.split(",").to_dict()

In [ ]:
# run differential expression per cluster
sc.tl.rank_genes_groups(adata, 'leiden_0.6', method='wilcoxon', key_added = "wilcoxon")

In [ ]:
# do gene set overlap to the groups in the gene list and top 300 DEGs.
import gseapy

gsea_res = dict()
pred = dict()

for cl in adata.obs['leiden_0.6'].cat.categories.tolist():
    print(cl)
    glist = sc.get.rank_genes_groups_df(adata, group=cl, key='wilcoxon')[
        'names'].squeeze().str.strip().tolist()
    enr_res = gseapy.enrichr(gene_list=glist[:300],
                             organism='Human',
                             gene_sets=gene_dict,
                             background=adata.shape[1],
                             cutoff=1)
    if enr_res.results.shape[0] == 0:
        pred[cl] = "Unass"
    else:
        enr_res.results.sort_values(
            by="P-value", axis=0, ascending=True, inplace=True)
        print(enr_res.results.head(2))
        gsea_res[cl] = enr_res
        pred[cl] = enr_res.results["Term"][0]

In [ ]:
# prediction per cluster
pred

In [ ]:
prediction = [pred[x] for x in adata.obs['leiden_0.6']]
adata.obs["GS_overlap_pred"] = prediction

sc.pl.umap(adata, color='GS_overlap_pred')

<div>

> **Discuss**
>
> As you can see, it agrees to some extent with the predictions from the
> methods above, but there are clear differences, which do you think
> looks better?

</div>

## Save data

We can finally save the object for use in future steps.

In [ ]:
adata.write_h5ad('data/covid/results/scanpy_covid_qc_dr_int_cl_ct-ctrl13.h5ad')

## Session info

```{=html}
<details>
```
```{=html}
<summary>
```
Click here
```{=html}
</summary>
```

In [ ]:
sc.logging.print_versions()

```{=html}
</details>
```